# Web Scraping - Strava

In [2]:
## Biblioteca necessárias

import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
)


## Iniciando uma instância do google chrome

In [3]:
# Pega o caminho do executável do ChromeDriver
service = Service(
    ChromeDriverManager().install()
)  # Corrigido, agora está criando o objeto Service corretamente.

# Usado para definir as preferências para o navegador Chrome
options = webdriver.ChromeOptions()

# Inicia-se uma instância do Google Chrome com as definições
driver = webdriver.Chrome(
    service=service, options=options
)  # Agora a passagem dos parâmetros está correta.

sleep(1)

### Entrando no Site - Strava
url = "https://www.strava.com/segments"
url_2 = "https://www.strava.com/segments/35231066"  # Segmento lamision35km2023
driver.get(url)

In [ ]:
# Login
usuario = "joaomarcos1296@gmail.com"
senha = "Jo@o1296"


campo_usuario = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(("xpath", '//*[@id="mobile-email"]'))
)
campo_usuario.send_keys(usuario)

campo_senha = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (
            "xpath",
            '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div/form/div[1]/div[2]/div/input',
        )
    )
)
campo_senha.send_keys(senha)

botao_login = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        ("xpath", '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div/form/div[2]/button')
    )
)
botao_login.click()
sleep(5)


# Acessa a Url Alvo
driver.get(url_2)
sleep(3)


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x483b03+62899]
	GetHandleVerifier [0x0x483b44+62964]
	(No symbol) [0x0x2b10f3]
	(No symbol) [0x0x2f980e]
	(No symbol) [0x0x2f9bab]
	(No symbol) [0x0x3425c2]
	(No symbol) [0x0x31e554]
	(No symbol) [0x0x33fd81]
	(No symbol) [0x0x31e306]
	(No symbol) [0x0x2ed670]
	(No symbol) [0x0x2ee4e4]
	GetHandleVerifier [0x0x6e4793+2556483]
	GetHandleVerifier [0x0x6dfd02+2537394]
	GetHandleVerifier [0x0x4aa2fa+220586]
	GetHandleVerifier [0x0x49aae8+157080]
	GetHandleVerifier [0x0x4a141d+184013]
	GetHandleVerifier [0x0x48ba68+95512]
	GetHandleVerifier [0x0x48bc10+95936]
	GetHandleVerifier [0x0x476b5a+9738]
	BaseThreadInitThunk [0x0x76025d49+25]
	RtlInitializeExceptionChain [0x0x772ed09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x772ed021+561]


## Função tabelas Corredores

### Tabelas dados voltas corredores

In [19]:
def tem_tabela_voltas():
    """
    Verifica se a aba 'Voltas' está disponível e clica nela se existir.
    Retorna True se conseguiu clicar, False caso contrário.
    """
    try:
        # Busca o elemento <li> que contém 'data-tracking-element="laps"'
        elemento_voltas = driver.find_element(
            By.CSS_SELECTOR, 'li[data-tracking-element="laps"]'
        )
        # Se encontrou o elemento, clica nele
        elemento_voltas.click()
        sleep(2)
        return True
    except:
        return False

In [13]:
def pega_tabela_voltas():
    """
    Extrai a tabela de voltas da página de atividades do Strava e salva como CSV.
    """

    # Encontrar a tabela
    table = driver.find_element(By.XPATH, '//*[@id="efforts-table"]')

    # Pegar os cabeçalhos
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # Pegar os dados da tabela
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Ignora o cabeçalho
        cols = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        data.append(cols)

    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data, columns=headers)

    return df


In [14]:
# Exemplo de uso:
# url_strava = "https://www.strava.com/activities/9680684368/laps"
# df_voltas = pega_tabela_voltas()
# print(df_voltas.head())  # Exibe as primeiras linhas do DataFrame

### Pega tabela de dados pagina inicial

In [15]:
def pega_tabela_inicial():
    """
    Extrai a tabela de voltas da página inicial do segmento do atleta do Strava
    """

    # Encontrar a tabela
    table = driver.find_element(By.XPATH, '//*[@id="run-efforts-table"]/section/table')

    # Pegar os cabeçalhos
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # Pegar os dados da tabela
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Ignora o cabeçalho
        cols = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        data.append(cols)

    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data, columns=headers)

    return df


# Exemplo de uso:
# url_strava = "https://www.strava.com/activities/9681673457#3128470765882939836"
# df_voltas = pega_tabela_inicial()
# print(df_voltas.head())  # Exibe as primeiras linhas do DataFrame

### Função navegação

In [16]:
def verifica_nomes_atletas():
    try:
        # Tente encontrar um elemento que contenha os nomes dos atletas
        driver.find_element(By.XPATH, '//*[@id="results"]/table/tbody/tr/td[2]/a')
        return True  # Se encontrar, retorna True (há nomes de atletas)
    except NoSuchElementException:
        return False


def verifica_num_pag():
    try:
        num_paginas = len(
            driver.find_elements(By.XPATH, '//*[@id="results"]/nav/ul/li')
        )
        if num_paginas == 0:
            num_paginas = 1
        else:
            num_paginas
    except:
        num_paginas = 1

    return num_paginas


def clicar_na_pagina(i):
    try:
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="results"]/nav/ul/li[{i + 1}]/a',
        ).click()
        sleep(1)
    except Exception:
        pass


def obter_nome_atleta(i, j):
    try:
        if i == 1 and j == 1:
            return driver.find_element(
                by=By.XPATH,
                value=f'//*[@id="results"]/table/tbody/tr[{j}]/td[2]/strong/a',
            ).text
        else:
            return driver.find_element(
                by=By.XPATH,
                value=f'//*[@id="results"]/table/tbody/tr[{j}]/td[2]/a',
            ).text
    except Exception:
        return None


def pega_nome_atletas():
    # Selecionando os dados dos corredores
    s = 1
    n = 26
    num_pag = verifica_num_pag()
    nomes_atletas = []

    # Ajustar o número de páginas
    if num_pag == 1:
        num_pag += 1
    else:
        num_pag -= 1

    for i in range(1, num_pag):
        clicar_na_pagina(i)
        for j in range(s, n):
            nome = obter_nome_atleta(i, j)
            if nome:
                nomes_atletas.append(nome)
            else:
                break

    return nomes_atletas

### Pegando dados atletas

In [8]:
"""def pega_dados_atletas():
    # Selecionando os dados dos corredores
    s = 1
    n = 1
    num_pag = verifica_num_pag()

    for i in range(1, num_pag-1):
        clicar_na_pagina(i)
        for j in range(s, n):
            try: # Verificando em qual aba está - 1 a 5 - caso não esteja na aba certa, clica-se na aba correta
                driver.find_element(
                    by=By.XPATH,
                    value=f'//*[@id="results"]/nav/ul/li[{i+1}]/a',
                    ).click()
                sleep(2)
                driver.find_element(
                    by = By.XPATH,
                    value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                    ).click() # Clicando no link do atleta
                driver.back()
                sleep(2)
        
            except Exception: # Caso ja esteja na aba correta, vai direto pro atleta
                try:
                    driver.find_element(
                        by = By.XPATH,
                        value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                        ).click() # Clicando no link do atleta
                    driver.back()
                    sleep(2)
                except Exception:
                    break"""

'def pega_dados_atletas():\n    # Selecionando os dados dos corredores\n    s = 1\n    n = 1\n    num_pag = verifica_num_pag()\n\n    for i in range(1, num_pag-1):\n        clicar_na_pagina(i)\n        for j in range(s, n):\n            try: # Verificando em qual aba está - 1 a 5 - caso não esteja na aba certa, clica-se na aba correta\n                driver.find_element(\n                    by=By.XPATH,\n                    value=f\'//*[@id="results"]/nav/ul/li[{i+1}]/a\',\n                    ).click()\n                sleep(2)\n                driver.find_element(\n                    by = By.XPATH,\n                    value = f\'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a\',\n                    ).click() # Clicando no link do atleta\n                driver.back()\n                sleep(2)\n        \n            except Exception: # Caso ja esteja na aba correta, vai direto pro atleta\n                try:\n                    driver.find_element(\n                        by = 

In [ ]:
######## Pega dados atçeta 2.0


def pega_dados_atletas(url_prova: str):
    """
    Percorre a lista de atletas, entra na página de cada um, coleta seu nome e extrai a tabela de voltas ou a tabela inicial.
    """
    driver.get(url_prova)
    s = 1
    n = 26
    num_pag = verifica_num_pag()
    nomes_atletas = pega_nome_atletas()

    dados_coletados = []
    print(f"Número de páginas: {num_pag}")
    for i in range(1, num_pag - 1):
        print(f"Acessando página {i} de {num_pag - 1}")
        sleep(2)
        clicar_na_pagina(i)
        sleep(2)
        for j in range(s, n):
            sleep(2)
            try:
                atleta_link = driver.find_element(
                    By.XPATH, f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a'
                )
                atleta_nome = (
                    nomes_atletas.pop(0) if nomes_atletas else f"Atleta_{i}_{j}"
                )
                atleta_link.click()
                sleep(2)

                try:
                    # Verifica se existe a aba "Voltas"
                    if tem_tabela_voltas():
                        sleep(5)
                        df = pega_tabela_voltas()
                    else:
                        sleep(5)
                        df = pega_tabela_inicial()

                    # Adiciona o nome do atleta à tabela
                    df.insert(0, "Nome Atleta", atleta_nome)
                    dados_coletados.append(df)
                    driver.back()
                    sleep(2)

                except Exception as e:
                    print(f"Atleta {atleta_nome} não tem tabela ou ocorreu um erro")
                    driver.back()
                    sleep(2)

                sleep(5)

            except Exception as e:
                print(
                    f"Erro ao acessar atleta {j} na página {i}, pulando para o próximo"
                )
                driver.back()  # Volta para a lista de atletas
                sleep(5)
                continue

    # Concatenar todos os DataFrames
    if dados_coletados:
        df_final = pd.concat(dados_coletados, ignore_index=True)
        df_final.to_csv("dados_atletas_2seg_p1.csv", index=False)
        print("Dados salvos em 'dados_atletas.csv'")
    else:
        print("Nenhum dado coletado.")

    return df_final if dados_coletados else None


In [ ]:
## Nova função


def pega_dados_atletas_corrigido(url_base_prova: str, nome_arquivo: str):
    """
    Percorre a lista de atletas de forma robusta, coletando os dados de cada um.

    Esta função navega página por página, coleta os links e nomes dos atletas
    de cada página antes de visitá-los, evitando erros de elementos "stale".
    """
    driver.get(url_base_prova)
    sleep(3)  # Espera inicial para a página carregar

    # 1. Descobrir o número de páginas
    num_paginas = verifica_num_pag()
    # Se a função retorna o número de 'li', e a última é 'next', ajustamos.
    # Se sua função já retorna o número correto, pode simplificar aqui.
    # No seu código original, num_pag era usado de forma um pouco confusa.
    # Vamos simplificar para ir de 1 até o número total de páginas.
    print(f"Total de páginas encontradas: {num_paginas}")

    dados_coletados = []

    # 2. Loop principal para cada PÁGINA de resultados
    for i in range(1, num_paginas + 1):
        # Constrói a URL da página específica. É mais confiável que clicar.
        # A maioria dos sites usa um parâmetro como '?page=X'
        url_pagina_atual = f"{url_base_prova}?page={i}"
        print(f"Acessando a página {i}: {url_pagina_atual}")
        driver.get(url_pagina_atual)
        sleep(3)  # Espera a página de resultados carregar

        # 3. Coletar todos os links e nomes de atletas ANTES de sair da página
        links_atletas = []
        nomes_atletas = []
        try:
            # Encontra todas as linhas (tr) da tabela de resultados
            linhas_tabela = driver.find_elements(
                By.XPATH, '//*[@id="results"]/table/tbody/tr'
            )

            for linha in linhas_tabela:
                try:
                    # Em cada linha, pega o link (href) e o nome do atleta
                    link_element = linha.find_element(By.XPATH, ".//td[3]/a")
                    nome_element = linha.find_element(By.XPATH, ".//td[2]/a")

                    links_atletas.append(link_element.get_attribute("href"))
                    nomes_atletas.append(nome_element.text)
                except NoSuchElementException:
                    # Ignora linhas que não são de atletas (cabeçalhos, etc.)
                    continue
        except Exception as e:
            print(f"Erro ao coletar links na página {i}: {e}")
            continue  # Pula para a próxima página

        print(f"Encontrados {len(links_atletas)} atletas na página {i}.")

        # 4. Loop para visitar cada LINK de atleta coletado
        for j, link in enumerate(links_atletas):
            atleta_nome = nomes_atletas[j]
            print(f"  -> Coletando dados de: {atleta_nome}")

            try:
                driver.get(link)
                sleep(2)  # Espera a página do atleta carregar

                df = None
                # Verifica se existe a aba "Voltas" e pega a tabela
                if tem_tabela_voltas():
                    # A função tem_tabela_voltas já clica, então só esperamos e pegamos a tabela
                    sleep(2)
                    df = pega_tabela_voltas()
                else:
                    # Se não tem 'Voltas', pega a tabela inicial
                    sleep(2)
                    df = pega_tabela_inicial()

                if df is not None:
                    # Adiciona o nome do atleta à tabela
                    df.insert(0, "Nome Atleta", atleta_nome)
                    dados_coletados.append(df)
                else:
                    print(f"     [Aviso] Nenhuma tabela encontrada para {atleta_nome}")

            except Exception as e:
                print(
                    f"     [Erro] Não foi possível processar o atleta {atleta_nome}. Erro: {e}"
                )
                # Não precisa usar driver.back(), pois o próximo passo do loop
                # é navegar para a URL do próximo atleta.
                continue

    # 5. Concatenar todos os DataFrames ao final
    if dados_coletados:
        df_final = pd.concat(dados_coletados, ignore_index=True)
        # Salve com um nome de sua escolha
        df_final.to_csv(nome_arquivo, index=False)
        print("\nProcesso concluído! Dados salvos em 'dados_strava_final.csv'")
        return df_final
    else:
        print("\nNenhum dado foi coletado.")
        return None


# --- Exemplo de como chamar a função ---
# url_da_prova = "https://www.strava.com/segments/SEU_SEGMENTO/leaderboard"
# df_completo = pega_dados_atletas_corrigido(url_da_prova)
# if df_completo is not None:
#    print(df_completo.head())

In [ ]:
df = pega_dados_atletas_corrigido(
    url_base_prova="https://www.strava.com/segments/35231066",
    nome_arquivo="dados_strava_final.csv",
)
if df is not None:
    print(df.head())

Total de páginas encontradas: 7
Acessando a página 1: https://www.strava.com/segments/35231066?page=1
Encontrados 24 atletas na página 1.
  -> Coletando dados de: Alexandre Santiago
  -> Coletando dados de: Caiê Alonso
  -> Coletando dados de: Mikael Martins
  -> Coletando dados de: Victor Caetano
  -> Coletando dados de: Breno Cordeiro
  -> Coletando dados de: EDU TRAILRUN ⛰️⛰️
  -> Coletando dados de: Bruno Reis
  -> Coletando dados de: Franklin von Haydin
  -> Coletando dados de: Ligia Almeida Fechio
  -> Coletando dados de: Gabriel Moreira
     [Erro] Não foi possível processar o atleta Gabriel Moreira. Erro: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="run-efforts-table"]/section/table"}
  (Session info: chrome=137.0.7151.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x903b03+62899]
	GetHandleVerifi

In [23]:
df.tail()

,Nome Atleta,KM,Ritmo,RAI,Elev.,Volta,Distância,Tempo,FC,Cadência
2998,Ana Cristina,NaN,5:22 /km,5:18 /km,-47 m,31,"1,00 km",5:22,116 bpm,NaN
2999,Ana Cristina,NaN,3:07 /km,3:13 /km,-85 m,32,"1,00 km",3:07,105 bpm,NaN
3000,Ana Cristina,NaN,3:07 /km,3:13 /km,-49 m,33,"1,00 km",3:07,93 bpm,NaN
3001,Ana Cristina,NaN,7:37 /km,7:37 /km,7 m,34,"1,00 km",7:37,102 bpm,NaN
3002,Ana Cristina,NaN,10:57 /km,12:12 /km,-1 m,35,"0,07 km",46 s,105 bpm,NaN


In [25]:
df_lido = pd.read_csv("dados_strava_final.csv")
df_lido.head()

,Nome Atleta,KM,Ritmo,RAI,Elev.,Volta,Distância,Tempo,FC,Cadência
0,Alexandre Santiago,1,4:43 /km,4:40 /km,-1 m,NaN,NaN,NaN,NaN,NaN
1,Alexandre Santiago,2,5:07 /km,3:41 /km,84 m,NaN,NaN,NaN,NaN,NaN
2,Alexandre Santiago,3,5:14 /km,4:12 /km,61 m,NaN,NaN,NaN,NaN,NaN
3,Alexandre Santiago,4,6:30 /km,4:14 /km,94 m,NaN,NaN,NaN,NaN,NaN
4,Alexandre Santiago,5,6:38 /km,4:46 /km,0 m,NaN,NaN,NaN,NaN,NaN


In [26]:
df["Nome Atleta"].nunique()


92

### Pega 'Sexo' Atletas

In [27]:
def pega_sexo_nomes(df):
    driver.refresh()
    for i in range(2, 4):
        # Clica no expandir os sexos
        driver.find_element(
            by=By.XPATH,
            value='//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/button',
        ).click()
        sleep(2)

        # Clica no sexo
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[{i}]/a',
        ).click()

        # Pega o texto do sexo
        sexo_texto = driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[{i}]/a',
        ).text

        sleep(3)

        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame(
                {
                    "sexo": [sexo_texto] * len(nomes_atletas),
                    "nome_atleta": nomes_atletas,
                }
            )
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [29]:
df_sexo = pd.DataFrame(columns=["sexo", "nome_atleta"])
df_sexo_atletas = pega_sexo_nomes(df_sexo)
df_sexo_atletas.head()

,sexo,nome_atleta
0,Homens,André Mapa
1,Homens,Alexandre Santiago
2,Homens,Caiê Alonso
3,Homens,Mikael Martins
4,Homens,Victor Caetano


### Pega Faixa Idade Atleta

In [30]:
def pega_faixas_nomes(df):
    driver.refresh()
    for i in range(2, 11):
        # Clica no expandir as idades
        driver.find_element(
            by=By.XPATH, value='//*[@id="premium-enhanced"]/ul/ul[2]/li[11]/button'
        ).click()
        sleep(2)

        # Clica na faixa idade
        driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).click()

        # Pega o texto da faixa etaria
        faixa_texto = driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).text

        sleep(3)

        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame(
                {
                    "faixa_etaria": [faixa_texto] * len(nomes_atletas),
                    "nome_atleta": nomes_atletas,
                }
            )
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [31]:
# Exemplo de uso
df_faixas_idades = pd.DataFrame(columns=["faixa_etaria", "nome_atleta"])
df_faixas_idades = pega_faixas_nomes(df_faixas_idades)
df_faixas_idades

,faixa_etaria,nome_atleta
0,20 a 24,Victor Caetano
1,20 a 24,Breno Cordeiro
2,20 a 24,Lucas Serra
3,20 a 24,Alex Junior Aj
4,25 a 34,Caiê Alonso
...,...,...
97,45 a 54,Viviane Montevequi
98,45 a 54,Ana Cristina
99,55 a 64,Pretinha Miguel
100,55 a 64,Magali Bellam


### Pega Peso Atletas

In [32]:
def pega_pesos_nomes(df):
    driver.refresh()
    for i in range(2, 10):
        # Clica no expandir os pesos
        driver.find_element(
            by=By.XPATH, value='//*[@id="premium-enhanced"]/ul/ul[3]/li[10]/button'
        ).click()
        sleep(2)

        # Clica na faixa peso
        driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[3]/li[{i}]/a'
        ).click()

        # Pega o texto da faixa etaria
        peso_texto = driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[3]/li[{i}]/a'
        ).text

        sleep(3)

        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame(
                {
                    "peso": [peso_texto] * len(nomes_atletas),
                    "nome_atleta": nomes_atletas,
                }
            )
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [33]:
# Exemplo de uso
df_pesos = pd.DataFrame(columns=["peso", "nome_atleta"])
df_pesos = pega_pesos_nomes(df_pesos)
df_pesos

,peso,nome_atleta
0,54 kg e abaixo,Breno Cordeiro
1,54 kg e abaixo,Ligia Almeida Fechio
2,54 kg e abaixo,Ferdinanda Alcântara
3,54 kg e abaixo,Chris 🏃🏼‍♀️🚴🏼‍♀️
4,54 kg e abaixo,Graci Lindenmayr #WRTrail
...,...,...
89,85 a 94 kg,João Paulo Souza
90,85 a 94 kg,Ana Cristina
91,95 kg a 104 kg,Flávio Rodrigues
92,95 kg a 104 kg,Tony Ortiz


## Juntando os DataFrames

## Segments

In [ ]:
url_1_2022 = "https://www.strava.com/segments/32632496"
url_2_2022 = "https://www.strava.com/segments/32632505"

In [ ]:
df_2022_1 = pega_dados_atletas_corrigido(
    url_base_prova=url_1_2022, nome_arquivo="dados_atletas_2022_1.csv"
)
df_2022_1.head()

### 2022 2